from https://colab.research.google.com/drive/115ba3EFCT0PvyXzFNv9E18QnKiyyjsm5?usp=sharing

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch
import ipywidgets as widgets

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")
base_model = LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native", device_map='auto', load_in_8bit=True, offload_folder='./offload/')

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)
local_llm = HuggingFacePipeline(pipeline=pipe)
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["instruction"])
llm_chain = LLMChain(prompt=prompt, llm=local_llm)

In [ ]:
question = "What is the capital of England?"
print(llm_chain.run(question))

In [ ]:
def on_button_clicked(b):
    with output:
        print(llm_chain.run(question))
output = widgets.Output()
question = widgets.Text()
button = widgets.Button(description='Chat')
button.on_click(on_button_clicked)
widgets.VBox([question, button, output])